In [1]:
pip install shapely

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import cv2
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab.patches import cv2_imshow
from scipy.spatial.transform import Rotation as R

## Constants

In [4]:
working_directory = '/content/drive/MyDrive/tern_project/'

os.chdir(working_directory)

In [5]:
## the convertion from cm to pixel in the drone image
s = 1.8

## the size of all the flag images
image_width = 1280
image_height = 720

## pixel location of the north camera on the drone image
north_cam_x = 11079
north_cam_y = 9341

## pixel location of the south camera on the drone image
south_cam_x = 13199
south_cam_y = 13490

north_cam_loc = np.array([north_cam_x,north_cam_y,0]).reshape(-1,1)
south_cam_loc = np.array([south_cam_x,south_cam_y,0]).reshape(-1,1)

## location in cm - north camera
cam_n = np.array( [0,0,266]).reshape(-1,1)

In [6]:
## finding the location of the south camera in the coordinate of the north camera (in cm)

## the constant to convert from pixel to cm
s_cm = 1/1.8

so1_u = s_cm*north_cam_x
so1_v = s_cm*north_cam_y

# with this matrix I can convert every location in pixel in the drone image to be in the coordinate system of the north camera
##(in cm)
Tr = np.transpose (np.array ([[s_cm,0,0],[0,s_cm,0],[-so1_u,-so1_v,1]]))

south_cam_pix = np.array([south_cam_x, south_cam_y,1])

south_cam_cm = np.dot(Tr , south_cam_pix)

osx = south_cam_cm[0]
osy = south_cam_cm[1]
z = 215

cam_s = np.array([osx,osy,z]).reshape(-1,1)
print (cam_s)

[[1177.77777778]
 [2305.        ]
 [ 215.        ]]


In [7]:
## the center of all of the flag images
center = [1280/2, 720/2]

def make_k(f,center):
  K = np.zeros((3,3))
  K[0,0] = f
  K[1,1] = f
  K[2,2] = 1
  K[0,2]= center[0]
  K[1,2] = center[1]
  return K

In [8]:
## calculate the p for the four corners
def calc_p (corners_df,r,k,cam_loc):
  ## cam_loc - the location in cm of the relevant camera

  u = corners_df['U'].values
  v = corners_df['V'].values
  one = np.ones (4)
  u_v = np.column_stack((u,v,one)).T

  ## (u,v,1) = r*k*(p-o)
  ## p-o = r-1 * k-1 * (u,v,1)
  p_o = (np.linalg.inv(r))@ np.linalg.inv(k) @u_v

  ## normalize the p-o by dividing the third component (z) to be az the minus z value (height) of the camera
  p_o = p_o/p_o[2]*-cam_loc[2]

  # p = p_o + cam (the z value qill be 0)
  p = p_o + cam_loc

  return (p)


Converting from cm to pixel location on the drone image

In [ ]:
def corner_loc_pix (s,cam_loc_pix, corner_real_loc):
  corner_pix_drone = (corner_real_loc*s) + cam_loc_pix.reshape (-1,1)

  corner_pix_drone = corner_pix_drone[0:2,:].T
  corner_pix_drone = corner_pix_drone.astype (np.int32)

  ## change the order of the points to be in a clockwise order
  line_2 =  corner_pix_drone[2,:].copy()
  line_3 = corner_pix_drone [3,:].copy()

  corner_pix_drone[3,:] = line_2
  corner_pix_drone[2,:] = line_3

  return (corner_pix_drone)

In [ ]:
# The drone image
drone_img = cv2.imread('drone_rotem/Shafiyot_True_Ortho.tif')

In [ ]:
file_path = 'Eyal/PTZ_modi_Cam_Values_191_mod.txt'

df_ptz_modi = pd.read_csv(file_path, sep='\t')

df_ptz_modi.head()

,ptz_num,pitch,yaw,zoom,f
0,24,-82.67,313.67,44.0,8738.100
1,25,-82.10,304.80,39.2,7840.932
2,26,-72.74,321.14,13.6,3056.036
3,27,-72.74,298.16,13.6,3056.036
4,28,-82.46,294.52,34.4,6943.764


In [ ]:
from shapely.geometry import Polygon

polygons = [];
ptz_cam_distance = {}

In [ ]:
def calc_centroid(polygon):
    corners = np.array(polygon)
    centroid_x = np.mean(corners[:, 0])
    centroid_y = np.mean(corners[:, 1])
    return (centroid_x, centroid_y)

def calc_distance(polygon, camera_position):
    polygon_center = calc_centroid(polygon)
    return math.sqrt((polygon_center[0] - camera_position[0]) ** 2 + (polygon_center[1] - camera_position[1]) ** 2)

In [ ]:
from shapely.geometry import Polygon

shelved_flags = [74, 75, 79]

for index, row in df_ptz_modi.iterrows():
    if row['ptz_num'] in shelved_flags:
        continue
    pitch = row['pitch']
    yaw = row['yaw']
    f = row['f']
    ptz_num = row['ptz_num']

    r = R.from_euler('zyx', [yaw, 0, pitch],degrees=True).as_matrix()
    k = make_k(f, center)


    # Create a DataFrame for the four corners of the flag images
    corners_df = pd.DataFrame({
        'Corner': ['Top Left', 'Top Right', 'Bottom Left', 'Bottom Right'],
        'U': [0, image_width - 1, 0, image_width - 1],
        'V': [0, 0, image_height - 1, image_height - 1]
    })

    # calculate the p (the location in cm) for the four corners
    p = calc_p(corners_df,r,k,cam_n)

    ## calculate the corners inpixel of the drone
    corners = corner_loc_pix(s,north_cam_loc, p)

    cv2.polylines(drone_img, [corners], isClosed=True, color=(0, 0, 255), thickness=7)

    number = str(int(ptz_num))
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 7
    font_color = (0, 0, 255)
    font_thickness = 20
    text_size = cv2.getTextSize(number, font, font_scale, font_thickness)[0]
    text_x = (corners[0, 0] + corners[1, 0] + corners [2,0] + corners [3,0]) // 4
    text_y = (corners[0, 1] + corners[1, 1] + corners [2,1] + corners [3,1]) // 4
    cv2.putText(drone_img, number, (text_x, text_y), font, font_scale, font_color, font_thickness, lineType=cv2.LINE_AA)

    polygon = [(corners[0][0], corners[0][1]), (corners[1][0], corners[1][1]), (corners[2][0], corners[2][1]), (corners[3][0], corners[3][1])]
    polygons.append({
        'coordinates': Polygon(polygon),
        'ptz_num': int(ptz_num)
    })

    ptz_cam_distance[int(ptz_num)] = calc_distance(polygon, (north_cam_x, north_cam_y))

scale_factor = 0.1

# Resize the image
resized_image = cv2.resize(drone_img, (0, 0), fx=scale_factor, fy=scale_factor)


cv2_imshow(resized_image)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
file_path = 'Eyal/PTZ_modi_Cam_Values_181_mod.txt'

df_ptz_modi_s = pd.read_csv(file_path, sep='\t')

df_ptz_modi_s.head()

,ptz_num,pitch,yaw,zoom,f
0,92,-84.89,327.10,45.6,9375.152
1,93,-85.29,319.70,50.8,10403.036
2,94,-83.80,312.23,50.8,10403.036
3,95,-83.60,305.35,50.8,10403.036
4,96,-83.60,298.23,50.8,10403.036


In [ ]:
shelved_flags = [120, 121, 137, 138]

for index, row in df_ptz_modi_s.iterrows():
    if row['ptz_num'] in shelved_flags:
        continue

    pitch = row['pitch']
    yaw = row['yaw']
    f = row['f']
    ptz_num = row ['ptz_num']

    r = R.from_euler('zyx', [yaw, 0, pitch],degrees=True).as_matrix()
    k = make_k(f, center)

    # Create a DataFrame for the four corners of the flag images
    corners_df = pd.DataFrame({
        'Corner': ['Top Left', 'Top Right', 'Bottom Left', 'Bottom Right'],
        'U': [0, image_width - 1, 0, image_width - 1],
        'V': [0, 0, image_height - 1, image_height - 1]
    })

    # calculate the p (the location in cm) for the four corners

    p = calc_p(corners_df,r,k,cam_n)

    ## calculate the corners inpixel of the drone
    corners = corner_loc_pix(s,south_cam_loc, p)


    cv2.polylines(drone_img, [corners], isClosed=True, color=(0, 0, 255), thickness=7)

    number = str(int(ptz_num))
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 7
    font_color = (0, 0, 255)
    font_thickness = 20
    text_size = cv2.getTextSize(number, font, font_scale, font_thickness)[0]
    text_x = (corners[0, 0] + corners[1, 0] + corners [2,0] + corners [3,0]) // 4
    text_y = (corners[0, 1] + corners[1, 1] + corners [2,1] + corners [3,1]) // 4
    cv2.putText(drone_img, number, (text_x, text_y), font, font_scale, font_color, font_thickness, lineType=cv2.LINE_AA)

    polygon = [(corners[0][0], corners[0][1]), (corners[1][0], corners[1][1]), (corners[2][0], corners[2][1]), (corners[3][0], corners[3][1])]
    polygons.append({
        'coordinates': Polygon(polygon),
        'ptz_num': int(number)
    })

    ptz_cam_distance[int(ptz_num)] = calc_distance(polygon, (south_cam_x, south_cam_y))

scale_factor = 0.1

# Resize the image
resized_image = cv2.resize(drone_img, (0, 0), fx=scale_factor, fy=scale_factor)

cv2_imshow(resized_image)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
len(polygons)

Function to find all overlays

Find overlays between three polygons

In [ ]:
from shapely.geometry import Polygon
from shapely.ops import unary_union

# Function to find all overlays
def find_overlays(polygons):
    # Find all pairwise overlays
    pairwise_overlays = {}
    for i in range(len(polygons)):
        for j in range(i + 1, len(polygons)):
            intersection = polygons[i]['coordinates'].intersection(polygons[j]['coordinates'])
            if not intersection.is_empty:
                pairwise_overlays[(polygons[i]['ptz_num'], polygons[j]['ptz_num'])] = intersection

    # Find overlays between three polygons
    triple_overlays = {}
    for i in range(len(polygons)):
        for j in range(i + 1, len(polygons)):
            for k in range(j + 1, len(polygons)):
                # Check if there is overlays between all pairs
                if(not ((polygons[i]['ptz_num'], polygons[j]['ptz_num']) in pairwise_overlays and
                   (polygons[i]['ptz_num'], polygons[k]['ptz_num']) in pairwise_overlays and
                   (polygons[j]['ptz_num'], polygons[k]['ptz_num']) in pairwise_overlays)):
                    continue

                intersection = polygons[i]['coordinates'].intersection(polygons[j]['coordinates']).intersection(polygons[k]['coordinates'])
                if not intersection.is_empty:
                    triple_overlays[(polygons[i]['ptz_num'], polygons[j]['ptz_num'], polygons[k]['ptz_num'])] = intersection
                    # Remove this part from pairwise overlays
                    pairwise_overlays[(polygons[i]['ptz_num'], polygons[j]['ptz_num'])] = \
                                            pairwise_overlays[(polygons[i]['ptz_num'], polygons[j]['ptz_num'])].difference(intersection)
                    pairwise_overlays[(polygons[j]['ptz_num'], polygons[k]['ptz_num'])] = \
                                            pairwise_overlays[(polygons[j]['ptz_num'], polygons[k]['ptz_num'])].difference(intersection)
                    pairwise_overlays[(polygons[i]['ptz_num'], polygons[k]['ptz_num'])] = \
                                            pairwise_overlays[(polygons[i]['ptz_num'], polygons[k]['ptz_num'])].difference(intersection)

    return pairwise_overlays, triple_overlays

pairwise_overlays, triple_overlays = find_overlays(polygons)

# Display results
print("Pairwise Overlays:")
print(pairwise_overlays.keys())

print("\nTriple Overlays:")
print(triple_overlays.keys())


Pairwise Overlays:
dict_keys([(24, 25), (24, 4), (24, 100), (24, 101), (25, 26), (25, 27), (25, 101), (26, 27), (26, 50), (26, 80), (26, 133), (26, 134), (27, 28), (27, 29), (27, 31), (27, 50), (27, 51), (29, 30), (29, 31), (30, 31), (30, 33), (31, 32), (31, 33), (31, 51), (32, 33), (32, 35), (32, 42), (32, 45), (32, 51), (32, 52), (33, 34), (33, 35), (33, 36), (34, 35), (35, 36), (35, 37), (35, 38), (36, 37), (38, 42), (39, 42), (40, 42), (40, 43), (41, 43), (42, 43), (43, 44), (44, 46), (45, 46), (45, 49), (45, 52), (46, 49), (47, 48), (47, 49), (48, 49), (48, 53), (48, 57), (49, 52), (49, 53), (49, 57), (50, 51), (50, 71), (50, 80), (50, 81), (50, 82), (50, 83), (50, 84), (50, 134), (50, 135), (51, 52), (51, 82), (51, 84), (52, 69), (52, 84), (52, 85), (53, 54), (53, 57), (54, 55), (54, 57), (4, 98), (4, 99), (4, 100), (4, 101), (4, 132), (4, 133), (55, 57), (55, 58), (56, 58), (56, 60), (57, 58), (57, 69), (58, 59), (58, 60), (58, 69), (58, 70), (59, 60), (59, 63), (59, 64), (59, 6

In [ ]:
from shapely.geometry import Polygon

for pairwise_overlay in pairwise_overlays:
    if not isinstance(pairwise_overlays[pairwise_overlay], Polygon):
        continue

    polygon = pairwise_overlays[pairwise_overlay]
    # Extracting exterior coordinates of the polygon
    exterior_coords = np.array(list(polygon.exterior.coords), dtype=np.int32)

    # exterior_coords = exterior_coords.reshape((-1, 1, 2))
    # cv2.fillPoly(drone_img, [exterior_coords], color=(0, 255, 0))
    cv2.polylines(drone_img, [exterior_coords], isClosed=True, color=(255, 255, 255), thickness=7)

scale_factor = 0.1

# Resize the image
resized_image = cv2.resize(drone_img, (0, 0), fx=scale_factor, fy=scale_factor)


cv2_imshow(resized_image)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from shapely.geometry import Polygon

for triple_overlay in triple_overlays:
    if not isinstance(triple_overlays[triple_overlay], Polygon):
        continue

    polygon = triple_overlays[triple_overlay]
    # Extracting exterior coordinates of the polygon
    exterior_coords = np.array(list(polygon.exterior.coords), dtype=np.int32)

    cv2.polylines(drone_img, [exterior_coords], isClosed=True, color=(0, 0, 0), thickness=7)

scale_factor = 0.1

# Resize the image
resized_image = cv2.resize(drone_img, (0, 0), fx=scale_factor, fy=scale_factor)


cv2_imshow(resized_image)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import json

# Your existing code that populates the redundant_areas dictionary
redundant_areas = {}

for overlay_flags in pairwise_overlays:
    if not isinstance(pairwise_overlays[overlay_flags], Polygon) or pairwise_overlays[overlay_flags].is_empty:
        continue

    polygon = pairwise_overlays[overlay_flags]

    min_flag = None
    min_dist = float('inf')

    # Get the closest flag to its camera
    for flag in overlay_flags:
        distance = ptz_cam_distance.get(flag, float('inf'))  # Use float('inf') for keys not found in the dictionary
        if distance < min_dist:
            min_dist = distance
            min_flag = flag

    for flag in overlay_flags:
        if flag == min_flag:
            continue

        redundant_areas.setdefault(flag, []).append(list(polygon.exterior.coords))

In [ ]:
file_path = './Eyal/RealCoordinatesCalculator/overlap_areas.json'
# Save redundant_areas as a JSON file with indentation
with open(file_path, 'w') as json_file:
    json.dump(redundant_areas, json_file, indent=4)

95
[[(7059.70113882861, 12650.570502605238), (7204.8187997287405, 12530.832787190757), (7205.946117274168, 12531.01267828843), (7060.0, 12651.0), (8908.300028664456, 11131.448988291671), (8905.0, 11128.0), (7059.70113882861, 12650.570502605238)]]


In [ ]:
import json

for overlay_flags in triple_overlays:
    if not isinstance(triple_overlays[overlay_flags], Polygon) or triple_overlays[overlay_flags].is_empty:
        continue

    polygon = triple_overlays[overlay_flags]

    min_flag = None
    min_dist = float('inf')

    # Get the closest flag to its camera
    for flag in overlay_flags:
        distance = ptz_cam_distance.get(flag, float('inf'))  # Use float('inf') for keys not found in the dictionary
        if distance < min_dist:
            min_dist = distance
            min_flag = flag

    for flag in overlay_flags:
        if flag == min_flag:
            continue

        redundant_areas.setdefault(flag, []).append(list(polygon.exterior.coords))

In [ ]:
file_path = './Eyal/RealCoordinatesCalculator/overlap_areas.json'
# Save redundant_areas as a JSON file with indentation
with open(file_path, 'w') as json_file:
    json.dump(redundant_areas, json_file, indent=4)